# Counting Comment Frequency

In this notebook, we take the subreddit data and count the number of comments in the first 3x minutes, where x is an integer.

In [ ]:
## import packages

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
## read data

## these are the posts
top_level = pd.read_pickle("WSBSubmissonsdata.pkl") 
## these are the comments under the posts
comm = pd.read_pickle("Commentsdata.pkl")

After reading the post 

In [ ]:
## creates a list parents_list of all the parent posts 

parents = top_level["id"]
parents_list = list(parents)

In [ ]:
## format list of posts so that post id matches post ids in comment data

for i in range(len(parents_list)):
    parents_list[i] = "t3_" + parents_list[i]

In [ ]:
## creates a list children indexed by the parent posts which assigns to each parent all the comments under it

children = {}
for parent in parents_list:
    child = comm[comm.parent_id == parent]
    children[parent] = child

In [ ]:
## creates an array comms_in_first indexed by integers i that assigns to each i the num of comments in 3(i+1) mins 

comms_in_first = []
for i in range(0,300):
    comms_in_first.append({})
    for parent in parents_list:
        times = list(children[parent]["time_created"])
        times.sort()
        start = int(top_level[top_level.id == parent[3:]]["time_created"])
        count = 0
        for t in times:
            if t - start <= 180 * (i+1):
                count += 1
        comms_in_first[i][parent[3:]] = count

In [ ]:
## for each id in data frame, add comms_in_first[i] minutes data

for i in range(0,300):
    name = 'comms_in_first' + str(i+1)
    top_level[name] = pd.Series(comms_in_first[i])

In [ ]:
## compute the correlation between comments in first 3i minutes and the total number of comments

correlations = []
for i in range(0,300):
    name = 'comms_in_first' + str(i+1)
    correlations.append(top_level[[name, 'num_comments']].astype(float).corr()[name]['num_comments'])

In [ ]:
corr_series = pd.Series(correlations)

In [ ]:
plt.figure(figsize = (8,8))

plt.scatter(pd.Series(range(0,300)), corr_series)
plt.xlabel("x", fontsize = 16)
plt.ylabel("Correlation coefficient between \n (# of comment in first 3x minutes) and (total number of comments)", fontsize = 16)

plt.show()